In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb

from data import read_data
from fe import preprocess, create_features
from utils import PurgedGroupTimeSeriesSplit

file_paths = {
    'price': '../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv',
    'list': '../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv',
    'financial': '../input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv'
}

dfs = create_features(
    preprocess(read_data(file_paths))
)

df = dfs['price'].merge(dfs['financial'], on=['SecuritiesCode', 'CurrentPeriodEndDate'], how='left')

/Users/tianjia.a.ni/workspace/jpx/src/data.py:15: DtypeWarning: Columns (14,15,16,17,18,19,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[name] = pd.read_csv(path)


In [11]:
[col for col in df.columns]

['RowId',
 'Date_x',
 'SecuritiesCode',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'AdjustmentFactor',
 'ExpectedDividend',
 'SupervisionFlag',
 'Target',
 'diff_high_low',
 'feat_pct20_Close',
 'feat_vl20_Close',
 'feat_rmr20_Close',
 'feat_wd20',
 'feat_pct40_Close',
 'feat_vl40_Close',
 'feat_rmr40_Close',
 'feat_wd40',
 'feat_pct60_Close',
 'feat_vl60_Close',
 'feat_rmr60_Close',
 'feat_wd60',
 'feat_pct120_Close',
 'feat_vl120_Close',
 'feat_rmr120_Close',
 'feat_wd120',
 'CumulativeAdjustmentFactor',
 'AdjustedClose',
 'CurrentPeriodEndDate',
 'DisclosureNumber',
 'DateCode',
 'Date_y',
 'DisclosedDate',
 'DisclosedTime',
 'DisclosedUnixTime',
 'TypeOfDocument',
 'TypeOfCurrentPeriod',
 'CurrentFiscalYearStartDate',
 'CurrentFiscalYearEndDate',
 'NetSales',
 'OperatingProfit',
 'OrdinaryProfit',
 'Profit',
 'EarningsPerShare',
 'TotalAssets',
 'Equity',
 'EquityToAssetRatio',
 'BookValuePerShare',
 'ResultDividendPerShare1stQuarter',
 'ResultDividendPerShare2ndQuarter',
 'Re

In [14]:
df = df.drop('Date_y', axis=1).rename(columns={'Date_x': 'Date'}, )